$$
\newcommand{\d}[2]{d_{#1}^{#2}}
\newcommand{\p}[2]{p_{#1}^{#2}}
\newcommand{\ceil}[1]{\left \lceil #1 \right \rceil }
\newcommand{\floor}[1]{\left \lfloor #1 \right \rfloor }
\newcommand{\dl}[2]{d_{#1}^{#2'}}
\begin{alignat}{3}
&\text{minimizar }&f(\d1-,\dl2-,\dl2+,\d3+; \p1+=5, \p2-=4, \p2+=10, \p3-=5, t_1 = 125, t2 = 12, t_3 = 55)\\
&\text{sujeito a }\\
&&D_1 \leq 6 \\
&&D_2 \geq 2 \\
&&D_3 \geq 1 \\
&&12D_1+9D_2+5D_3+\d1-\geq125\\
&&D_1+\frac{2}{5}D_2+\frac{4}{5}D_3+\dl2--\dl2+=12\\
&&5D_1+5D_2+8D_3-\d3+ \leq 55\\
&&\d{i}{\pm} \geq 0\\
&&\dl2\pm \in \mathbb{N_0}
\end{alignat}
$$

fica (Handbook of Critical Issues in Goal Programming, pag37 - pag39)
$$
\newcommand{\d}[2]{d_{#1}^{#2}}
\newcommand{\p}[2]{p_{#1}^{#2}}
\newcommand{\ceil}[1]{\left \lceil #1 \right \rceil }
\newcommand{\floor}[1]{\left \lfloor #1 \right \rfloor }
\newcommand{\dl}[2]{d_{#1}^{#2'}}
\newcommand{\dll}[2]{d_{#1}^{#2''}}
\begin{alignat}{3}
&\text{minimizar }&5\dl1- +4\dll2- + 10\dll2+ 5\dl3+  \\
&\text{sujeito a }\\
&&D_1 \leq 6 \\
&&D_2 \geq 2 \\
&&D_3 \geq 1 \\
&&9.6D_1+7.2D_2+4D_3+\dl1-\geq100\\
&&8.(3)D_1+3.(3)D_2+6.(6)D_3+\dll2--\dll2+=100\\
&&9.(09)D_1+9.(09)D_2+14.(54)D_3-\dl3+ \leq 100\\
&&\d{i}{\pm} \geq 0\\
&&\dl2\pm \in \mathbb{N_0} \\
&&
\end{alignat} \\
\text{,sendo } \dll{i}{\pm} = \frac{100 \cdot \dl{i}{\pm}}{t_i}
$$

In [ ]:
from pulp import *
from math import *

In [ ]:
d = {
    (i,pm):LpVariable(f"d{i}{'mais' if pm == '+' else 'menos'}", lowBound=0, cat = 'Integer' if i == 2 else 'Continuous') 
        for i in range(1,4) for pm in ["+", "-"]
}
D = {i: LpVariable(f"D{i}", lowBound=0) for i in range(1, 4)}
p = {
    (1, "-"): 5,
    (2, "-"): 4,
    (2, "+"): 10,
    (3, "+"): 5
}
t = {
    1:125,
    2:12,
    3:55
}

def unline(dlinha, t):
    return (dlinha*t)/100

In [ ]:
modelo = LpProblem("modelo4", LpMinimize)
modelo += (D[1] <= 6, "maximo de D1")
modelo += (D[2] >= 2, "minimo de D2")
modelo += (D[3] >= 1, "minimo de D3")
modelo += (9.6*D[1] + 7.2*D[2] + 4*D[3]+d[1, "-"] >= 100, "meta de alcance de lucro")
modelo += ((8 + 1/3)*D[1] + (3 + 1/3) * D[2] + (6 + 2/3) * D[3] + d[2,"-"] - d[2,"+"] == 100, "meta de manter o nivel de mao de obra")
modelo += ((1/11)*D[1] + (1/11)*D[2] + (14 + 6/11)*D[3] - d[3, "+"] <= 100, "meta de travar o investimento")
modelo += 5*d[1, "-"] + 4*d[2, "-"] + 10*d[2, "+"] + 5*d[3, "+"]
modelo

modelo4:
MINIMIZE
5*d1menos + 10*d2mais + 4*d2menos + 5*d3mais + 0
SUBJECT TO
maximo_de_D1: D1 <= 6

minimo_de_D2: D2 >= 2

minimo_de_D3: D3 >= 1

meta_de_alcance_de_lucro: 9.6 D1 + 7.2 D2 + 4 D3 + d1menos >= 100

meta_de_manter_o_nivel_de_mao_de_obra: 8.33333333333 D1 + 3.33333333333 D2
 + 6.66666666667 D3 - d2mais + d2menos = 100

meta_de_travar_o_investimento: 0.0909090909091 D1 + 0.0909090909091 D2
 + 14.5454545455 D3 - d3mais <= 100

VARIABLES
D1 Continuous
D2 Continuous
D3 Continuous
d1menos Continuous
0 <= d2mais Integer
0 <= d2menos Integer
d3mais Continuous

In [ ]:
print("status: ",modelo.solve())
print("valor do objetivo:",value(modelo.objective))
print("variáveis:")
for i in modelo.variables():
    print("   ", i.name, "=", i.varValue)

status:  1
valor do objetivo: 0.0
variáveis:
    D1 = 6.0
    D2 = 13.0
    D3 = 1.0
    d1menos = 0.0
    d2mais = 0.0
    d2menos = 0.0
    d3mais = 0.0


Parece dar os mesmos valores de desvios porque a meta 1 já está na mesma unidade que a meta 3, e a meta 2 como é inteira, e a penalização é pequena ao de empregar mais pessoas, o modelo decidiu como importante mesmo quando usando os desvios percentuais. O lucro a longo prazo atinge sempre o pretendido, mas prevê-se que o investimento terá que ser maior. Isto faz sentido sendo que D1 contribui de forma drástica para o lucro comparado com o investimento necessário e a mão de obra que precisa (sendo que o modelo achou necessário usar sempre o máximo desse doce), ainda que este é o que precisa de mais mão de obra; tanto que o modelo recomenda empregar mais para este lucro se observar.